In [1]:
# !pip install -qU pymupdf4llm transformers sentence-transformers datasets bitsandbytes
# !pip install orjson==3.9.15
# !pip install -qU langchain chromadb langchain-chroma langchain_community
# !pip install langchain-huggingface
# !pip install rank_bm25

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import pymupdf4llm
import re
from tqdm.notebook import tqdm
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever, ContextualCompressionRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
import torch
import os
import pandas as pd
import pandas as pd
import numpy as np
import unicodedata

In [4]:
# def split_documents(file_path_list):
#     documents = []
#     for data_file in tqdm(file_path_list, total = len(file_path_list)):
#         print(data_file)
#         pdf = pymupdf4llm.to_markdown(data_file) # , write_images=True, image_path=project_path+'/processed/image/pymupdf4llm/' 이미지 나중에
#         pdf =  re.sub(r'(?<!\n)\n\n(?!\n)', '\n', pdf) # 약간의 전처리
#         documents.append(Document(page_content=pdf, metadata={"source": data_file}))

#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1600, chunk_overlap=50, separators=["\n\n-----\n", "\n\n", "\n", " ", ""]) # 한 페이지를 가득 채우는 table 하나가 1300~1400 정도
#     split_documents = text_splitter.split_documents(documents)
#     return split_documents

def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    # PDF 파일 열기
    pdf = pymupdf4llm.to_markdown(file_path)
    pdf =  re.sub(r'(?<!\n)\n\n(?!\n)', '\n', pdf)
    doc = [Document(page_content=pdf, metadata={"source": file_path})]
    # pdf를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_documents(doc)
    # Document 객체 리스트 생성
    return chunk_temp

def create_vector_db(document, embeddings, file_path):
    db = Chroma.from_documents(
    documents=document,
    embedding=embeddings, persist_directory=file_path)
    return db

def create_pdf_databases(base_directory, embeddings, chunk_size, chunk_overlap):
    """딕셔너리에 pdf명을 키로 해서 DB, retriever 저장"""
    pdf_databases = {}
    for p in tqdm(os.listdir(base_directory), desc="Creating Vectordb for each PDF"):
        print(p)
        doc = process_pdf(base_directory + p, chunk_size, chunk_overlap)
        # print(doc)

        # base_directory에서 train_source 또는 test_source 디렉토리를 찾아서 파일 경로 결정
        if base_directory.split('/')[-2] == 'train_source':
            file_path = base_directory[:base_directory.find('train_source/')]
        elif base_directory.split('/')[-2] == 'test_source':
            file_path = base_directory[:base_directory.find('test_source/')]

        # vectordb의 저장 경로 설정
        vectordb_path = file_path + 'pymupdf4llm/' + p[:p.find('.pdf')]

        # 경로에 파일이 이미 존재하는지 확인
        if os.path.exists(vectordb_path):
            print(f"Vector DB already exists at {vectordb_path}, loading existing DB...")
            vectordb = Chroma(persist_directory=vectordb_path, embedding_function=embeddings)
        else:
            print(f"Creating new Vector DB at {vectordb_path}...")
            vectordb = create_vector_db(doc, embeddings, vectordb_path)

        # retriever = vectordb.as_retriever(search_kwargs={"k": 3})
        pdf_databases[unicodedata.normalize('NFC', p.split('.pdf')[0])] = {
            'db': vectordb,
            # 'retriever': retriever,
            'doc': doc
        }

    return pdf_databases

## Multiple Vector for each pdf

In [5]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [6]:
# base_directory = '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/train_source/'
base_directory = '../train_source/'
pdf_databases = create_pdf_databases(base_directory, bge_embeddings, chunk_size=500, chunk_overlap=50)

Creating Vectordb for each PDF:   0%|          | 0/16 [00:00<?, ?it/s]

1-1 2024 주요 재정통계 1권.pdf
Vector DB already exists at ../pymupdf4llm/1-1 2024 주요 재정통계 1권, loading existing DB...
2024 나라살림 예산개요.pdf
Vector DB already exists at ../pymupdf4llm/2024 나라살림 예산개요, loading existing DB...
2024년도 성과계획서(총괄편).pdf
Vector DB already exists at ../pymupdf4llm/2024년도 성과계획서(총괄편), loading existing DB...
「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf
Vector DB already exists at ../pymupdf4llm/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》, loading existing DB...
「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf
Vector DB already exists at ../pymupdf4llm/「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》, loading existing DB...
고용노동부_내일배움카드(일반).pdf
Vector DB already exists at ../pymupdf4llm/고용노동부_내일배움카드(일반), loading existing DB...
고용노동부_조기재취업수당.pdf
Vector DB already exists at ../pymupdf4llm/고용노동부_조기재취업수당, loading existing DB...
고용노동부_청년일자리창출지원.pdf
Vector DB already exists at ../pymupdf4llm/고용노동부_청년일자리창출지원, loading existing DB...
국토교통부_민간임대(융자).pdf
Vector DB already exists at ../pymupdf4llm/국토교통부_민간임대(융자), loading existing DB...
국토교통부

In [7]:
pdf_databases

{'1-1 2024 주요 재정통계 1권': {'db': <langchain_chroma.vectorstores.Chroma at 0x214899e9a30>,
  'doc': [Document(metadata={'source': '../train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='한국재정정보원-2024 주요 재정통계\nⅠ권 1장\n･\n표지 면지\n\n\n-----\n2024 주요 재정통계Ⅰ\n･\n기획 조정 재정정보분석센터 연훈수 선임연구위원\n작 성 재정정보분석센터 김선옥 부연구위원, 전규식 연구원, 김지선 과장\n문 의 한국재정정보원 재정정보분석센터(02-6908-8200)\n이 책은 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.'),
   Document(metadata={'source': '../train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='-----\n###### 발 [/ ] 간 [/ ] 사\n한국재정정보원은 재정정보화 플랫폼인 디지털예산회계시스템(dBrain+)과 국고보조\n금통합관리시스템(e나라도움)을 안정적으로 운영하고, 예･결산 등 시스템 내에 축적된\n다양한 재정정보를 분석하여 재정정책의 수립 운용을 지원하고 있습니다.\n2018년부터 발간한 ｢주요 재정통계｣는 디지털예산회계시스템의 재정정보를 예산체\n계에 따른 시계열 통계로 구성･제공하고 있습니다. 아울러 중앙-지방정부 등 여러 곳에\n산재되어 있는 재정통계를 수록하여 재정분석의 기초자료로 활용될 수 있도록 하였습니'),
   Document(metadata={'source': '../train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='다. 또한 단순 정보전달에 그치지 않고, 일반 국민도 쉽게 이해할 수 있도록 재정통계의\n가독성을 높이고자 노력하였습니다.\n특히, 올해부터는 한권으로 제공되던 ｢주요 재정통계｣를 Ⅰ, Ⅱ

In [8]:
# train_df = pd.read_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/train.csv')
train_df = pd.read_csv('../train.csv')
train_df

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [9]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

In [10]:
# train_df[:2]

In [11]:
# train_df['Answer'][0]

In [12]:
# train_df.loc[495:]

In [13]:
# train_df[495:]['Answer'].values

In [14]:
# for _, row in tqdm(train_df[495:].iterrows(), total=len(train_df[495:]), desc = "Creating prompt template"):
#     retriever = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['db'].as_retriever(search_kwargs={"k":5})
#     compressor = CrossEncoderReranker(model=reranker_model, top_n=2)
#     doc = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['doc']
#     bm25_retriever = BM25Retriever.from_texts([d.page_content for d in doc], metadatas=[d.metadata for d in doc])
#     bm25_retriever.k = 2

#     ensemble_retriever = EnsembleRetriever(
#     retrievers=[bm25_retriever, retriever],
#     weights=[0.5, 0.5],
#     # search_type="mmr",
#     )

#     # 문서 압축 검색기 초기화
#     compression_retriever = ContextualCompressionRetriever(
#         base_compressor=compressor, base_retriever=ensemble_retriever
#     )
#     docs = compression_retriever.invoke(row['Question'])
#     print(docs)

In [15]:
def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

model_id = "rtzr/ko-gemma-2-9b-it"

# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False


prompt_list = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc = "Creating prompt template"):
    retriever = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['db'].as_retriever(search_kwargs={"k":5})
    doc = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['doc']
    bm25_retriever = BM25Retriever.from_texts([d.page_content for d in doc], metadatas=[d.metadata for d in doc])
    bm25_retriever.k = 2

    ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5],
    # search_type="mmr",
    )

    # 상위 3개의 문서 선택
    compressor = CrossEncoderReranker(model=reranker_model, top_n=3)

    # 문서 압축 검색기 초기화
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    docs = compression_retriever.invoke(row['Question'])
    context = format_docs(docs)
    question = row['Question']
    answer = row['Answer']
    template = f"""
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.

    답변: {answer}{tokenizer.eos_token}
    """

    prompt_list.append(template)

Creating prompt template:   0%|          | 0/496 [00:00<?, ?it/s]

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


In [16]:
prompt_list[:5]

['\n    다음 정보를 바탕으로 질문에 답하세요:\n    -----\nⅠ.\n주요 재정통계\n## 01 [재정체계]\n▸\n중앙정부 재정체계는 예산(일반･특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계\n21개, 기금 68개로 구성\n∙\n2024년 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성\n∙\n2024년 기금 지출은 49개 사업성기금 81.2조원, 6개 사회보험성기금 107.1조원, 5개 계정성기금\n30.1조원으로 구성\n[그림 1-1] 재정지출 구조(2024년 예산 총지출 기준)\n\n\n주: 괄호 안은 총계 기준 예산액을 의미\n자료: 디지털예산회계시스템\n\n\n-----\n2024 주요 재정통계 | 2024 Fiscal Statistics\n▸\n예산은 ｢국가재정법｣에 근거해 정부가 편성하고 국회가 심의･의결로 확정한 재정지출계획을 의미하며,\n일반회계와 특별회계로 구분\n\n-----\nⅠ.\n주요 재정통계\n(2) 국가채무\n▸\n2024년 예산 기준, 국가채무는 1,196.2조원을 기록할 것으로 전망됨\n∙\n국가채무는 중앙정부 채무(1,163.4조원)와 지방정부 순채무(32.8조원)로 구성되며, 적자성 채무가\n66.2%(792.4조원)를 차지\n[그림 3-8] 국가채무 추이 (단위: 조원, %)\n\n\n중앙･지방정부 채무 추이\n･\n적자성 금융성 채무 추이\n자료: 기획재정부재정정보공개시스템(openfiscaldata.go.kr)\n\n\n-----\n2024 주요 재정통계 | 2024 Fiscal Statistics\n[표 3-43] 국가채무 추이 (단위: 조원, %)\n중앙정부 지방정부\n국가채무 GDP대비,\n중앙정부 국고채무 순채무\n(A+B) % 국채 차입금\n채무(A) 부담행위 (B)\n\n-----\n2024 주요 재정통계 | 2024 Fiscal Statistics\n참고 12대 분야별 재원 배분과 국가재정운용계획\n\n\n▸\n정부는 프로그램 예산 체계

In [17]:
train_df['prompt'] = prompt_list

In [18]:
train_df

,SAMPLE_ID,Source,Source_path,Question,Answer,prompt
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년...",\n 다음 정보를 바탕으로 질문에 답하세요:\n -----\nⅠ.\n주요...
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7...",\n 다음 정보를 바탕으로 질문에 답하세요:\n -----\nⅠ.\n주요...
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정...",\n 다음 정보를 바탕으로 질문에 답하세요:\n ∙\n기금은 예산과 구분...
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별...",\n 다음 정보를 바탕으로 질문에 답하세요:\n -----\nⅠ.\n주요...
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216...",\n 다음 정보를 바탕으로 질문에 답하세요:\n [표 1-1] 일반회계 ...
...,...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10...",\n 다음 정보를 바탕으로 질문에 답하세요:\n 로 재화와 서비스를 제공...
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...,"\n 다음 정보를 바탕으로 질문에 답하세요:\n -----\n재정, 그땐..."
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ...",\n 다음 정보를 바탕으로 질문에 답하세요:\n 월 10일 「한미원조협정...
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ...",\n 다음 정보를 바탕으로 질문에 답하세요:\n -----\n해외재정동향...


In [20]:
train_df.to_csv('finetuning_prompt.csv', index = False)